In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/alexnetbtc_mnist/pytorch/default/1/alexnet_btc_mnist.pth


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from PIL import Image, ImageFilter
from tqdm import tqdm

# --- Device Setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Hyperparameters ---
learning_rate = 0.01
momentum = 0.9
weight_decay = 1e-4
batch_size = 128
num_epochs_total = 50
epochs_per_blur = 10
blur_levels = [8, 4, 2, 1, 0]

# --- Gaussian Blur Wrapper ---
class GaussianBlur:
    def __init__(self, sigma):
        self.sigma = sigma

    def __call__(self, img):
        return img.filter(ImageFilter.GaussianBlur(self.sigma))

# --- DataLoader Generator ---
def get_dataloader(blur_level, batch_size):
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=3),
        transforms.CenterCrop((256, 256)),
        transforms.Resize((224, 224)),
        transforms.Lambda(lambda img: GaussianBlur(blur_level)(img)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

# --- Test Loader (no blur) ---
test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.CenterCrop((256, 256)),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# --- Compute Validation Accuracy ---
def evaluate_accuracy(model, val_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    return acc

# --- Training Loop ---
def train_model(model, criterion, optimizer, train_loader, val_loader, device, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
        for inputs, labels in pbar:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(loss=loss.item())

        # Live accuracy after each epoch
        acc = evaluate_accuracy(model, val_loader, device)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss:.4f}, Accuracy: {acc:.2f}%")

    return model

# --- BTC Trainer ---
def adjust_blur_and_train(model):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                          momentum=momentum, weight_decay=weight_decay)

    current_epoch = 0
    for blur in blur_levels:
        if current_epoch >= num_epochs_total:
            break
        print(f"\n--- Training with blur level {blur} ---")
        epochs_to_train = min(epochs_per_blur, num_epochs_total - current_epoch)
        train_loader = get_dataloader(blur, batch_size)
        model = train_model(model, criterion, optimizer, train_loader, test_loader, device, num_epochs=epochs_to_train)
        current_epoch += epochs_to_train
    return model

# --- Main Execution ---
if __name__ == '__main__':
    alexnet = models.alexnet(pretrained=False)
    alexnet.classifier[6] = nn.Linear(4096, 10)  # 10-class output for MNIST
    alexnet = alexnet.to(device)

    print("Starting BTC Training for AlexNet...")
    trained_model = adjust_blur_and_train(alexnet)

    # Save model
    torch.save(trained_model.state_dict(), "alexnet_btc_mnist.pth")

Starting BTC Training for AlexNet...

--- Training with blur level 8 ---


Epoch 1/10, Loss: 1079.5029, Accuracy: 11.35%


Epoch 2/10, Loss: 1079.3695, Accuracy: 11.35%


Epoch 3/10, Loss: 1079.3554, Accuracy: 11.35%


Epoch 4/10, Loss: 1079.3566, Accuracy: 11.35%


In [2]:
import torch
import torch.nn as nn
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

# --- Device Setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Test Transform (No Blur) ---
test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.CenterCrop((256, 256)),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# --- Load Test Set ---
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# --- Load BTC-Trained AlexNet ---
model = models.alexnet(pretrained=False)
model.classifier[6] = nn.Linear(4096, 10)  # 10-class MNIST
model.load_state_dict(torch.load("/kaggle/input/alexnetbtc_mnist/pytorch/default/1/alexnet_btc_mnist.pth"))
model = model.to(device)
model.eval()

# --- Evaluate Clean Accuracy ---
correct = 0
total = 0
with torch.no_grad():
    pbar = tqdm(test_loader, desc="Evaluating")
    for inputs, labels in pbar:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"\n✅ Final Clean Accuracy: {accuracy:.2f}%")

100%|██████████| 9.91M/9.91M [00:00<00:00, 35.7MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.06MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 10.0MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.23MB/s]
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Evaluating: 100%|██████████| 79/79 [00:24<00:00,  3.17it/s]


✅ Final Clean Accuracy: 98.07%


In [3]:
!pip install torchattacks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.0/142.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
import torchattacks
from tqdm import tqdm

# ---------------- PGD Attack Setup ----------------
pgd_eps = 0.001         # Changeable
pgd_alpha = 2/255
pgd_steps = 40

pgd_attack = torchattacks.PGD(model, eps=pgd_eps, alpha=pgd_alpha, steps=pgd_steps)

# ---------------- PGD Adversarial Accuracy Function ----------------
def adversarial_test_pgd(attack, loader):
    model.eval()
    correct = 0
    total = 0
    
    for inputs, labels in tqdm(loader, desc=f"Adversarial Test (PGD)"):
        inputs, labels = inputs.to(device), labels.to(device)
        adv_inputs = attack(inputs, labels)
        outputs = model(adv_inputs)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
    
    acc = correct / total
    print(f"📊 PGD Accuracy (ε={pgd_eps}): {acc:.4f}")
    return acc

# ---------------- Run PGD Evaluation ----------------
pgd_acc = adversarial_test_pgd(pgd_attack, test_loader)

Adversarial Test (PGD): 100%|██████████| 79/79 [04:27<00:00,  3.38s/it]

📊 PGD Accuracy (ε=0.001): 0.1820


In [12]:

# --- Define PGD Attack ---
attack = torchattacks.PGD(model, eps=0.1, alpha=2/225, steps=40)
print("Running PGD attack with ε = 0.1, α = 0.01, steps = 40")

# --- Evaluate Adversarial Accuracy ---
correct = 0
total = 0
for inputs, labels in tqdm(test_loader, desc="Evaluating under PGD"):
    inputs, labels = inputs.to(device), labels.to(device)
    adv_images = attack(inputs, labels)
    outputs = model(adv_images)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

adv_accuracy = 100 * correct / total
print(f"\n⚠️  PGD Adversarial Accuracy (ε=0.1): {adv_accuracy:.2f}%")

Running PGD attack with ε = 0.1, α = 0.01, steps = 40


Evaluating under PGD: 100%|██████████| 79/79 [04:27<00:00,  3.39s/it]


⚠️  PGD Adversarial Accuracy (ε=0.1): 17.39%


In [4]:
import torchattacks
from tqdm import tqdm

# ---------------- CW Attack Setup ----------------
cw_c = 1e-3
cw_kappa = 0
cw_steps = 100
cw_lr = 0.01

cw_attack = torchattacks.CW(model, c=cw_c, kappa=cw_kappa, steps=cw_steps, lr=cw_lr)

# ---------------- CW Adversarial Accuracy Function ----------------
def adversarial_test_cw(attack, loader):
    model.eval()
    correct = 0
    total = 0

    for inputs, labels in tqdm(loader, desc=f"Adversarial Test (CW)"):
        inputs, labels = inputs.to(device), labels.to(device)
        adv_inputs = attack(inputs, labels)
        outputs = model(adv_inputs)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    acc = correct / total
    print(f"📊 CW Accuracy (c={cw_c}, kappa={cw_kappa}): {acc:.4f}")
    return acc

# ---------------- Run CW Evaluation ----------------
cw_acc = adversarial_test_cw(cw_attack, test_loader)

Adversarial Test (CW): 100%|██████████| 79/79 [14:14<00:00, 10.82s/it]

📊 CW Accuracy (c=0.001, kappa=0): 0.1801
